In [ ]:
# !pip install cassandra-driver
# !pip install prettytable

In [1]:
import pandas as pd
import cassandra
from cassandra.query import UNSET_VALUE
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable
import datetime

In [42]:

# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/product_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\qycer\iCloudDrive\INFO7275\Project


In [43]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

7299


In [129]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('product_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['productid','asins','brand','categories','colors','dateAdded',
                     'dateUpdated','dimension','imageURLs','manufacturer',
                     'manufacturerNumber','ProductName','primaryCategories',
                     'reviews_date','reviews_doRecommend','reviews_numHelpful',
                     'reviews_rating','reviews_text','reviews_title',
                     'reviews_username','weight'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow(row)

In [2]:
# check the number of rows in your csv file
with open('product_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

7300


In [2]:
#################### CONNECT TO DATASTAX CASSANDRA ########################
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
        'secure_connect_bundle': 'C:/Users/qycer/\iCloudDrive/INFO7275/\Project/secure-connect-team6-projecct.zip'
}
auth_provider = PlainTextAuthProvider('team6', 'info7275')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.0.684


In [3]:
############# SET THE KEYSPACES ########
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)

In [12]:
########### READ DATA FROM CSV ###############
#rawdf = pd.read_csv('product_datafile_new.csv')

In [10]:
#Convert date string into DATAFRAME
df = rawdf
df['Reviews_ID'] = df.index+1
#for i in range(df.shape[0]):
#    if pd.isnull(df['reviews_date'][i]):
#        continue
#    else:
#        df.loc[i, 'reviews_date'] = datetime.datetime.strptime(df['reviews_date'][i][:10],"%Y-%m-%d")

####VERY IMPORTANT: CONVERT TO NONE CORRECTLY)
#df1 = df.where(pd.notnull(df), None)
df1 = df.replace(np.nan, None)

In [11]:
#df[df.duplicated(subset = ['productid', 'brand', 'ProductName', 'reviews_username'], keep = 'first')]
df2 = df1.drop_duplicates(subset = ['productid', 'brand', 'ProductName', 'reviews_username'], keep = 'first')

In [171]:
df2.to_csv('productdata.csv', index=False)

In [4]:

############### new data source using ####################


df2 = pd.read_csv('productdata.csv')

#CQL DO NOT EXECUTE HERE \\
CREATE TABLE IF NOT EXISTS customer_reviews (productid text,
                                             ProductName text,
                                             reviews.date date,
                                             reviews.doRecommend boolean,
                                             reviews.numHelpful int,
                                             reviews.rating float,
                                             reviews.text text,
                                             reviews.title text,
                                             reviews.username text,
                                             PRIMARY KEY (productid), reviews.rating, reviews.date)

## Table: customer_reviews

In [193]:
create_reviews = """CREATE TABLE IF NOT EXISTS customer_reviews (
                                                reviews_id int,
                                                productid text,
                                                ProductName text,
                                                brand text,
                                                reviews_date timestamp,
                                                reviews_doRecommend boolean,
                                                reviews_numHelpful float,
                                                reviews_rating float,
                                                reviews_text text,
                                                reviews_title text,
                                                reviews_username text,
                                                PRIMARY KEY((reviews_id, productid, brand, ProductName, reviews_username),
                                                reviews_rating
                                                ))"""
try:
    rows = session.execute(create_reviews)
except Exception as e:
    print(e)

In [194]:
query_insert_customer_reviews = """INSERT INTO customer_reviews(
                                                reviews_id,
                                                productid,
                                                ProductName,
                                                brand,
                                                reviews_date,
                                                reviews_doRecommend,
                                                reviews_numHelpful,
                                                reviews_rating,
                                                reviews_text,
                                                reviews_title,
                                                reviews_username
                                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""


for index, row in df2.iterrows():
    try:
        session.execute(query_insert_customer_reviews,
                        (row["Reviews_ID"], row["productid"], row["ProductName"], row['brand'], row["reviews_date"],
                         row["reviews_doRecommend"], row["reviews_numHelpful"],
                         row["reviews_rating"], row["reviews_text"], row["reviews_title"],
                         row["reviews_username"]))   
    except Exception as e:
        print(e)
        
#### NEED TO DEAL WITH EMPTY DATA #####(CONVERT THEM TO "NONE")

In [181]:
#If in a mess, just drop the entire table!
session.execute("drop table customer_reviews")

In [168]:
#CHECK HOW MANY WE HAVE INSERTED
select_reviews_info = """SELECT count(*)
                            FROM customer_reviews"""

try:
    rows = session.execute(select_reviews_info)
    print (rows[0].count)
except Exception as e:
    print(e)

6983


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys


## Table: product_details

In [18]:
df3.shape

(50, 22)

In [45]:
df3 = df2.drop_duplicates(subset = ['ProductName', 'productid', 'dateAdded', 'primaryCategories'], keep = 'first')

In [196]:
create_product_details = """CREATE TABLE IF NOT EXISTS product_details (
                                                productid text,
                                                ProductName text,
                                                brand text,
                                                categories text,
                                                colors text,
                                                dateAdded timestamp,
                                                dateUpdated timestamp,
                                                manufacturer text,
                                                manufacturerNumber text,
                                                primaryCategories text,
                                                PRIMARY KEY((ProductName, productid, dateAdded, primaryCategories),
                                                colors,brand
                                                ))"""
try:
    rows = session.execute(create_product_details)
except Exception as e:
    print(e)

In [198]:
query_insert_product_details = """INSERT INTO product_details (
                                                productid,
                                                ProductName,
                                                brand,
                                                categories,
                                                colors,
                                                dateAdded,
                                                dateUpdated,
                                                manufacturer,
                                                manufacturerNumber,
                                                primaryCategories                                            
                                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

#with open(file, encoding = 'utf8') as f:
#    csvreader = csv.reader(f)
#    next(csvreader) # skip header

for index, row in df3.iterrows():
    try:
        session.execute(query_insert_product_details,
                        (row["productid"], row["ProductName"], row['brand'], row["categories"],
                         row["colors"], row["dateAdded"], row["dateUpdated"],
                         row["manufacturer"], row["manufacturerNumber"], 
                         row["primaryCategories"]))   
    except Exception as e:
        print(e)
        
#### NEED TO DEAL WITH EMPTY DATA #####(CONVERT THEM TO "NONE")

In [195]:
#If in a mess, just drop the entire table!
session.execute("drop table product_details")

In [199]:
#CHECK HOW MANY WE HAVE INSERTED
select_product_details_info = """SELECT count(*)
                            FROM product_details"""

try:
    rows = session.execute(select_product_details_info)
    print (rows[0].count)
except Exception as e:
    print(e)

50


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys


## Table: product

In [53]:
create_product = """CREATE TABLE IF NOT EXISTS product (
                                                productid text,
                                                ProductName text,
                                                brand text,
                                                categories text,
                                                colors text,
                                                primaryCategories text,
                                                PRIMARY KEY(brand, productid)
                                                
                                                )"""
try:
    rows = session.execute(create_product)
except Exception as e:
    print(e)

In [54]:
query_insert_product = """INSERT INTO product (
                                                productid,
                                                ProductName,
                                                brand,
                                                categories,
                                                colors,
                                                primaryCategories                                            
                                                ) VALUES (%s, %s, %s, %s, %s, %s)"""

#with open(file, encoding = 'utf8') as f:
#    csvreader = csv.reader(f)
#    next(csvreader) # skip header

for index, row in df3.iterrows():
    try:
        session.execute(query_insert_product,
                        (row["productid"], row["ProductName"], row['brand'], row["categories"],
                         row["colors"], row["primaryCategories"]))   
    except Exception as e:
        print(e)

In [52]:
#If in a mess, just drop the entire table!
session.execute("drop table product")

In [55]:
#CHECK HOW MANY WE HAVE INSERTED
select_product_info = """SELECT count(*)
                            FROM product"""

try:
    rows = session.execute(select_product_info)
    print (rows[0].count)
except Exception as e:
    print(e)

50


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys


## Table: Transaction

In [22]:
create_transaction = """CREATE TABLE IF NOT EXISTS transaction (
                                                reviews_username text,
                                                productid text,
                                                dateAdded timestamp,
                                                dateUpdated timestamp,
                                                PRIMARY KEY((reviews_username, productid), dateAdded
                                                )
                                                )"""
try:
    rows = session.execute(create_transaction)
except Exception as e:
    print(e)

In [24]:
query_insert_transaction = """INSERT INTO transaction (
                                                reviews_username,
                                                productid,
                                                dateAdded,
                                                dateUpdated
                                                ) VALUES (%s, %s, %s, %s)"""



for index, row in df2.iterrows():
    try:
        session.execute(query_insert_transaction,
                        (row["reviews_username"],row["productid"],  row['dateAdded'], row["dateUpdated"]))   
    except Exception as e:
        print(e)

In [5]:
#CHECK HOW MANY WE HAVE INSERTED
select_transaction_info = """SELECT count(*)
                            FROM transaction"""

try:
    rows = session.execute(select_transaction_info)
    print (rows[0].count)
except Exception as e:
    print(e)

6983


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys


# Table： Users

In [7]:
df_user = df2.drop_duplicates(subset = ['reviews_username', 'Reviews_ID'], keep = 'first')

In [18]:
df_user.head()

,productid,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,imageURLs,manufacturer,...,primaryCategories,reviews_date,reviews_doRecommend,reviews_numHelpful,reviews_rating,reviews_text,reviews_title,reviews_username,weight,Reviews_ID
0,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,https://i5.walmartimages.com/asr/2a41f6f0-844e...,Microsoft,...,Electronics,2015-12-04T00:00:00.000Z,True,0.0,5.0,"This keyboard is very easy to type on, but the...",Love the fingerprint reader,JNH1,1.1 pounds,1
1,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,https://i5.walmartimages.com/asr/2a41f6f0-844e...,Microsoft,...,Electronics,2015-12-04T00:00:00.000Z,True,0.0,4.0,It's thin and light. I can type pretty easily ...,Nice,Appa,1.1 pounds,2
2,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,https://i5.walmartimages.com/asr/2a41f6f0-844e...,Microsoft,...,Electronics,2015-12-04T00:00:00.000Z,True,0.0,4.0,I love the new design the keys are spaced well...,New,Kman,1.1 pounds,3
3,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,https://i5.walmartimages.com/asr/2a41f6f0-844e...,Microsoft,...,Electronics,2015-12-01T00:00:00.000Z,True,0.0,5.0,Attached easily and firmly. Has a nice feel. A...,Nice keyboard,UpstateNY,1.1 pounds,4
4,AVpf3txeLJeJML43FN82,B0168YIWSI,Microsoft,"Electronics,Computers,Computer Accessories,Key...",Black,2015-11-13T12:28:09Z,2018-01-29T02:15:13Z,11.6 in x 8.5 in x 0.19 in,https://i5.walmartimages.com/asr/2a41f6f0-844e...,Microsoft,...,Electronics,2015-11-28T00:00:00.000Z,True,0.0,5.0,"Our original keyboard was okay, but did not ha...",Nice improvement,Glickster,1.1 pounds,5


In [24]:
create_users = """CREATE TABLE IF NOT EXISTS users (
                                                reviews_username text,
                                                reviews_id int,
                                                ProductName text,
                                                reviews_rating float,
                                                reviews_doRecommend boolean,
                                                reviews_title text,
                                                reviews_text text,
                                                
                                                PRIMARY KEY((reviews_username), reviews_id)
                                                
                                                )"""
try:
    rows = session.execute(create_users)
except Exception as e:
    print(e)

In [25]:
query_insert_users = """INSERT INTO users (
                                                reviews_username,
                                                reviews_id,
                                                ProductName,
                                                reviews_rating,
                                                reviews_doRecommend,
                                                reviews_title,
                                                reviews_text
                                                ) VALUES (%s, %s, %s, %s, %s, %s, %s)"""



for index, row in df_user.iterrows():
    try:
        session.execute(query_insert_users,
                        (row["reviews_username"],row["Reviews_ID"], row["ProductName"],
                        row["reviews_rating"],row["reviews_doRecommend"], row["reviews_title"], row["reviews_text"]))   
    except Exception as e:
        print(e)

In [26]:
#CHECK HOW MANY WE HAVE INSERTED
select_users_info = """SELECT count(*)
                            FROM users"""

try:
    rows = session.execute(select_users_info)
    print (rows[0].count)
except Exception as e:
    print(e)

6983


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys


In [23]:
session.execute("drop table users")

# Table: Manufacturer

In [27]:
df_manu = df2.drop_duplicates(subset = ['manufacturer', 'manufacturerNumber'], keep = 'first')

In [29]:
create_manufacturer = """CREATE TABLE IF NOT EXISTS manufacturer (
                                                manufacturer text,
                                                ManufacturerNumber text,
                                                
                                                PRIMARY KEY((manufacturer), ManufacturerNumber)
                                                
                                                )"""
try:
    rows = session.execute(create_manufacturer)
except Exception as e:
    print(e)

In [31]:
query_insert_manufacturer = """INSERT INTO manufacturer (
                                                manufacturer,
                                                ManufacturerNumber
                                                ) VALUES (%s, %s)"""



for index, row in df_manu.iterrows():
    try:
        session.execute(query_insert_manufacturer,
                        (row["manufacturer"],row["manufacturerNumber"]))   
    except Exception as e:
        print(e)

In [32]:
#CHECK HOW MANY WE HAVE INSERTED
select_manu_info = """SELECT count(*)
                            FROM manufacturer"""

try:
    rows = session.execute(select_manu_info)
    print (rows[0].count)
except Exception as e:
    print(e)

50


C:\Users\qycer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  import sys
